In [1]:
import os
import pandas as pd
import sklearn.metrics
import numpy as np
import optuna
import warnings
warnings.filterwarnings('ignore')

/opt/homebrew/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
TRIALS = 250
TOP_N = 1

In [23]:
def load_performance_metrics(dataset, gnn, llm, method, aggregation, display_information = False):
  
  study_name = f'{dataset}-{gnn}-{llm.replace("/", "-")}-{method}-{aggregation}'
  storage = f'sqlite:///../../pipelines/optuna_studies/{study_name}.db'
  study = optuna.load_study(study_name = study_name, storage = storage)
  
  study_df = study.trials_dataframe()

  study_df = study_df[study_df['value'] >= 0.0].sort_values(by = 'number', ascending = True)
  columns = ['number', 'value'] + [x for x in study_df.columns if x.startswith('params_')] + [x for x in study_df.columns if x.startswith('user_attrs_')]

  top_trials = study_df[columns].head(TRIALS).sort_values(by = ['value', 'user_attrs_validation_loss', 'user_attrs_training_loss'], ascending = [False, True, True]).head(TOP_N)# .head(3) for R8, BART-large due to exceptions
  
  performance = list()
  #for trial in [x for x in os.listdir(f'../../outputs/{dataset}-{gnn}-{llm}/{method}/{aggregation}') if os.path.isdir(f'../../outputs/{dataset}-{gnn}-{llm}/{method}/{aggregation}/{x}')]:
  for _, row in top_trials.iterrows():
    
    if display_information:
      print(row)
    trial = row['number']

    for random_state in [x for x in os.listdir(f'../../outputs/{dataset}-{gnn}-{llm}/{method}/{aggregation}/{trial}') if os.path.isdir(f'../../outputs/{dataset}-{gnn}-{llm}/{method}/{aggregation}/{trial}/{x}')]:
      df = pd.read_csv(f'../../outputs/{dataset}-{gnn}-{llm}/{method}/{aggregation}/{trial}/{random_state}/predictions.csv')
      for split in ['validation', 'test']:
        df_split = df[df['split'] == split]
        accuracy = sklearn.metrics.accuracy_score(df_split['real'], df_split['prediction'])
        f1_score = sklearn.metrics.f1_score(df_split['real'], df_split['prediction'], average = 'macro')
        precision = sklearn.metrics.precision_score(df_split['real'], df_split['prediction'], average = 'macro')
        recall = sklearn.metrics.recall_score(df_split['real'], df_split['prediction'], average = 'macro')
        performance.append((trial, random_state, split, accuracy, f1_score, precision, recall))
  return pd.DataFrame(performance, columns = ['trial', 'random_state', 'split', 'accuracy', 'f1_score', 'precision', 'recall'])

In [4]:
def remove_trials_with_exceptions(df):
  return df.groupby(['trial']).filter(lambda x : len(x) >= 10 * 2).reset_index(drop = True)

In [5]:
def get_best_results(dataset, gnn, llm, method, aggregation, target_metric):
  df = load_performance_metrics(dataset = dataset, gnn = gnn, llm = llm, method = method, aggregation = aggregation, display_information = True)
  df = remove_trials_with_exceptions(df)
  df_aggregated = df.groupby(['trial', 'split']).agg({
    'accuracy' : ['mean', 'std', 'max'],
    'f1_score' : ['mean', 'std'],
    'precision' : ['mean', 'std'],
    'recall' : ['mean', 'std'],
  }).reset_index()
  df_aggregated.columns = df_aggregated.columns.map('_'.join).str.strip('_')
  
  best_trial_number = df_aggregated.sort_values(by = ['split', f'{target_metric}_mean', f'{target_metric}_std'], ascending = [True, False, True]).reset_index(drop = True).iloc[0]['trial']

  best_trial = df_aggregated[df_aggregated['trial'] == best_trial_number]

  print(best_trial_number)
  print(
    'Accuracy:',
    '{:.2f}'.format(np.round(best_trial[best_trial['split'] == 'test']['accuracy_mean'].values[0] * 100, decimals = 2)),
    '±', 
    '{:.2f}'.format(np.round(best_trial[best_trial['split'] == 'test']['accuracy_std'].values[0] * 100, decimals = 2)),
    ',',
    '{:.2f}'.format(np.round(best_trial[best_trial['split'] == 'test']['accuracy_max'].values[0] * 100, decimals = 2)),
  )

# RQ 4

_______________

## BERT-base, Grouped

In [6]:
get_best_results(dataset = 'SST-2', gnn = 'GATv2', llm = 'google-bert-bert-base-uncased', method = 'Grouped', aggregation = 'No_Aggregation', target_metric = 'accuracy')

number                                     221
value                                  0.91055
params_attention_heads                       7
params_attention_pooling_operation         min
params_balanced_loss                      True
params_batch_size                           67
params_beta_0                         0.836906
params_beta_1                         0.988719
params_dropout_rate                   0.469089
params_early_stopping_patience              23
params_embedding_pooling_operation         max
params_epochs                               69
params_epsilon                             0.0
params_global_pooling                      max
params_hidden_dimension                    238
params_learning_rate                  0.000592
params_number_of_hidden_layers               2
params_plateau_divider                       6
params_plateau_patience                     19
params_threshold                      0.574329
params_weight_decay                   0.000125
user_attrs_ep

In [7]:
get_best_results(dataset = 'Ohsumed', gnn = 'GATv2', llm = 'google-bert-bert-base-uncased', method = 'Grouped', aggregation = 'No_Aggregation', target_metric = 'accuracy')

number                                     291
value                                 0.657566
params_attention_heads                      14
params_attention_pooling_operation         max
params_balanced_loss                     False
params_batch_size                           40
params_beta_0                         0.821612
params_beta_1                         0.985736
params_dropout_rate                   0.397687
params_early_stopping_patience              18
params_embedding_pooling_operation        mean
params_epochs                              171
params_epsilon                        0.000007
params_global_pooling                      max
params_hidden_dimension                    241
params_learning_rate                  0.000142
params_left_stride                           0
params_number_of_hidden_layers               2
params_plateau_divider                       9
params_plateau_patience                     10
params_right_stride                         64
params_thresh

In [8]:
get_best_results(dataset = 'R8', gnn = 'GATv2', llm = 'google-bert-bert-base-uncased', method = 'Grouped', aggregation = 'No_Aggregation', target_metric = 'accuracy')

number                                     220
value                                 0.966294
params_attention_heads                       6
params_attention_pooling_operation        mean
params_balanced_loss                     False
params_batch_size                           92
params_beta_0                         0.895508
params_beta_1                         0.987714
params_dropout_rate                   0.552551
params_early_stopping_patience              20
params_embedding_pooling_operation         min
params_epochs                               64
params_epsilon                             0.0
params_global_pooling                      max
params_hidden_dimension                    125
params_learning_rate                  0.000316
params_left_stride                          32
params_number_of_hidden_layers               2
params_plateau_divider                       9
params_plateau_patience                     23
params_right_stride                        128
params_thresh

In [9]:
#get_best_results(dataset = 'RTSP', gnn = 'GATv2', llm = 'google-bert-bert-base-uncased', method = 'Grouped', aggregation = 'No_Aggregation', target_metric = 'accuracy')

## BERT-base, Surrogate

In [10]:
get_best_results(dataset = 'SST-2', gnn = 'GATv2', llm = 'google-bert-bert-base-uncased', method = 'Surrogate', aggregation = 'No_Aggregation', target_metric = 'accuracy')

number                                     255
value                                 0.912844
params_attention_heads                      16
params_attention_pooling_operation         max
params_balanced_loss                      True
params_batch_size                           77
params_beta_0                         0.834165
params_beta_1                         0.981367
params_dropout_rate                   0.454022
params_early_stopping_patience              15
params_embedding_pooling_operation         max
params_epochs                              197
params_epsilon                        0.000002
params_global_pooling                     mean
params_hidden_dimension                    196
params_learning_rate                  0.001156
params_number_of_hidden_layers               3
params_plateau_divider                       5
params_plateau_patience                     11
params_threshold                       0.48984
params_weight_decay                   0.000007
user_attrs_ep

In [11]:
get_best_results(dataset = 'Ohsumed', gnn = 'GATv2', llm = 'google-bert-bert-base-uncased', method = 'Surrogate', aggregation = 'No_Aggregation', target_metric = 'accuracy')

number                                     417
value                                 0.655114
params_attention_heads                       9
params_attention_pooling_operation         max
params_balanced_loss                     False
params_batch_size                           36
params_beta_0                         0.815749
params_beta_1                         0.981366
params_dropout_rate                    0.35046
params_early_stopping_patience              16
params_embedding_pooling_operation         min
params_epochs                               59
params_epsilon                         0.00008
params_global_pooling                      max
params_hidden_dimension                    105
params_learning_rate                  0.000311
params_left_stride                         256
params_number_of_hidden_layers               2
params_plateau_divider                      10
params_plateau_patience                     12
params_right_stride                         64
params_thresh

In [12]:
get_best_results(dataset = 'R8', gnn = 'GATv2', llm = 'google-bert-bert-base-uncased', method = 'Surrogate', aggregation = 'No_Aggregation', target_metric = 'accuracy')

number                                     215
value                                 0.957508
params_attention_heads                       8
params_attention_pooling_operation         max
params_balanced_loss                     False
params_batch_size                          110
params_beta_0                         0.816872
params_beta_1                         0.996901
params_dropout_rate                   0.348761
params_early_stopping_patience              20
params_embedding_pooling_operation         max
params_epochs                               72
params_epsilon                        0.000002
params_global_pooling                      max
params_hidden_dimension                     37
params_learning_rate                  0.000097
params_left_stride                          32
params_number_of_hidden_layers               2
params_plateau_divider                       2
params_plateau_patience                     23
params_right_stride                          0
params_thresh

In [13]:
#get_best_results(dataset = 'RTSP', gnn = 'GATv2', llm = 'google-bert-bert-base-uncased', method = 'Surrogate', aggregation = 'No_Aggregation', target_metric = 'accuracy')

## BART-base, Grouped

In [14]:
get_best_results(dataset = 'SST-2', gnn = 'GATv2', llm = 'facebook-bart-base', method = 'Grouped', aggregation = 'No_Aggregation', target_metric = 'accuracy')

number                                     141
value                                 0.925459
params_attention_heads                      12
params_attention_pooling_operation         max
params_balanced_loss                     False
params_batch_size                          121
params_beta_0                         0.846054
params_beta_1                         0.998926
params_dropout_rate                   0.442588
params_early_stopping_patience              21
params_embedding_pooling_operation         max
params_epochs                               63
params_epsilon                        0.000002
params_global_pooling                      max
params_hidden_dimension                     58
params_learning_rate                  0.003539
params_number_of_hidden_layers               4
params_plateau_divider                       4
params_plateau_patience                     12
params_threshold                      0.662727
params_weight_decay                   0.000078
user_attrs_ep

In [15]:
get_best_results(dataset = 'Ohsumed', gnn = 'GATv2', llm = 'facebook-bart-base', method = 'Grouped', aggregation = 'No_Aggregation', target_metric = 'accuracy')

number                                     224
value                                 0.629495
params_attention_heads                       4
params_attention_pooling_operation         max
params_balanced_loss                     False
params_batch_size                          105
params_beta_0                         0.810466
params_beta_1                         0.987975
params_dropout_rate                   0.416937
params_early_stopping_patience              20
params_embedding_pooling_operation        mean
params_epochs                              161
params_epsilon                             0.0
params_global_pooling                     mean
params_hidden_dimension                    141
params_learning_rate                  0.001984
params_left_stride                          32
params_number_of_hidden_layers               1
params_plateau_divider                       5
params_plateau_patience                     14
params_right_stride                         64
params_thresh

In [16]:
get_best_results(dataset = 'R8', gnn = 'GATv2', llm = 'facebook-bart-base', method = 'Grouped', aggregation = 'No_Aggregation', target_metric = 'accuracy')

number                                     170
value                                 0.968568
params_attention_heads                       6
params_attention_pooling_operation         max
params_balanced_loss                     False
params_batch_size                          163
params_beta_0                         0.848785
params_beta_1                         0.994063
params_dropout_rate                   0.324637
params_early_stopping_patience              20
params_embedding_pooling_operation         max
params_epochs                              116
params_epsilon                             0.0
params_global_pooling                      max
params_hidden_dimension                    132
params_learning_rate                  0.000106
params_left_stride                         128
params_number_of_hidden_layers               2
params_plateau_divider                       4
params_plateau_patience                     12
params_right_stride                          0
params_thresh

In [17]:
get_best_results(dataset = 'RTSP', gnn = 'GATv2', llm = 'facebook-bart-base', method = 'Grouped', aggregation = 'No_Aggregation', target_metric = 'accuracy')

number                                     235
value                                 0.877637
params_attention_heads                      14
params_attention_pooling_operation         min
params_balanced_loss                     False
params_batch_size                           89
params_beta_0                         0.846446
params_beta_1                         0.987113
params_dropout_rate                   0.588638
params_early_stopping_patience              13
params_embedding_pooling_operation         max
params_epochs                               69
params_epsilon                             0.0
params_global_pooling                      max
params_hidden_dimension                    233
params_learning_rate                  0.000161
params_number_of_hidden_layers               0
params_plateau_divider                       5
params_plateau_patience                     13
params_threshold                      0.408957
params_weight_decay                   0.000037
user_attrs_ep

## BART-large, Grouped

In [18]:
get_best_results(dataset = 'SST-2', gnn = 'GATv2', llm = 'facebook-bart-large', method = 'Grouped', aggregation = 'No_Aggregation', target_metric = 'accuracy')

number                                     111
value                                 0.952982
params_attention_heads                       7
params_attention_pooling_operation         min
params_balanced_loss                     False
params_batch_size                          188
params_beta_0                         0.833149
params_beta_1                         0.994217
params_dropout_rate                   0.569644
params_early_stopping_patience              13
params_embedding_pooling_operation        mean
params_epochs                               73
params_epsilon                             0.0
params_global_pooling                      max
params_hidden_dimension                     49
params_learning_rate                  0.001239
params_number_of_hidden_layers               3
params_plateau_divider                       4
params_plateau_patience                     20
params_threshold                      0.603626
params_weight_decay                   0.000005
user_attrs_ep

In [19]:
get_best_results(dataset = 'Ohsumed', gnn = 'GATv2', llm = 'facebook-bart-large', method = 'Grouped', aggregation = 'No_Aggregation', target_metric = 'accuracy')

number                                     184
value                                 0.676073
params_attention_heads                       9
params_attention_pooling_operation         max
params_balanced_loss                     False
params_batch_size                           75
params_beta_0                         0.825653
params_beta_1                         0.989955
params_dropout_rate                   0.468633
params_early_stopping_patience              25
params_embedding_pooling_operation        mean
params_epochs                              133
params_epsilon                        0.000007
params_global_pooling                     mean
params_hidden_dimension                    241
params_learning_rate                  0.001416
params_left_stride                          64
params_number_of_hidden_layers               1
params_plateau_divider                       2
params_plateau_patience                     21
params_right_stride                         64
params_thresh

In [22]:
get_best_results(dataset = 'R8', gnn = 'GATv2', llm = 'facebook-bart-large', method = 'Grouped', aggregation = 'No_Aggregation', target_metric = 'accuracy')

number                                     255
value                                 0.980578
params_attention_heads                      16
params_attention_pooling_operation        mean
params_balanced_loss                      True
params_batch_size                           54
params_beta_0                         0.885474
params_beta_1                         0.983331
params_dropout_rate                   0.575229
params_early_stopping_patience              20
params_embedding_pooling_operation         max
params_epochs                              144
params_epsilon                        0.000001
params_global_pooling                      max
params_hidden_dimension                     77
params_learning_rate                  0.000019
params_left_stride                          64
params_number_of_hidden_layers               3
params_plateau_divider                       7
params_plateau_patience                     19
params_right_stride                        128
params_thresh

In [24]:
get_best_results(dataset = 'RTSP', gnn = 'GATv2', llm = 'facebook-bart-large', method = 'Grouped', aggregation = 'No_Aggregation', target_metric = 'accuracy')

number                                     168
value                                 0.911392
params_attention_heads                       7
params_attention_pooling_operation         max
params_balanced_loss                     False
params_batch_size                          169
params_beta_0                         0.811166
params_beta_1                         0.989678
params_dropout_rate                   0.539217
params_early_stopping_patience              23
params_embedding_pooling_operation         max
params_epochs                              121
params_epsilon                        0.000002
params_global_pooling                      max
params_hidden_dimension                    140
params_learning_rate                  0.000214
params_number_of_hidden_layers               0
params_plateau_divider                       7
params_plateau_patience                     25
params_threshold                       0.59677
params_weight_decay                   0.000338
user_attrs_ep

## BART-large, Surrogate

In [25]:
get_best_results(dataset = 'SST-2', gnn = 'GATv2', llm = 'facebook-bart-large', method = 'Surrogate', aggregation = 'No_Aggregation', target_metric = 'accuracy')

number                                     126
value                                 0.952982
params_attention_heads                       6
params_attention_pooling_operation         min
params_balanced_loss                     False
params_batch_size                          201
params_beta_0                         0.880565
params_beta_1                         0.983501
params_dropout_rate                   0.544881
params_early_stopping_patience              21
params_embedding_pooling_operation         min
params_epochs                              192
params_epsilon                        0.000007
params_global_pooling                      max
params_hidden_dimension                    125
params_learning_rate                  0.007342
params_number_of_hidden_layers               1
params_plateau_divider                       2
params_plateau_patience                     13
params_threshold                      0.564438
params_weight_decay                   0.000143
user_attrs_ep

In [26]:
get_best_results(dataset = 'Ohsumed', gnn = 'GATv2', llm = 'facebook-bart-large', method = 'Surrogate', aggregation = 'No_Aggregation', target_metric = 'accuracy')

number                                     320
value                                  0.66319
params_attention_heads                       5
params_attention_pooling_operation         min
params_balanced_loss                     False
params_batch_size                           37
params_beta_0                         0.877274
params_beta_1                         0.984732
params_dropout_rate                   0.397249
params_early_stopping_patience              25
params_embedding_pooling_operation         max
params_epochs                              191
params_epsilon                        0.000054
params_global_pooling                      max
params_hidden_dimension                    184
params_learning_rate                  0.000789
params_left_stride                         256
params_number_of_hidden_layers               0
params_plateau_divider                      10
params_plateau_patience                     17
params_right_stride                        128
params_thresh

In [27]:
get_best_results(dataset = 'R8', gnn = 'GATv2', llm = 'facebook-bart-large', method = 'Surrogate', aggregation = 'No_Aggregation', target_metric = 'accuracy')

number                                     240
value                                 0.972184
params_attention_heads                      15
params_attention_pooling_operation         min
params_balanced_loss                     False
params_batch_size                          117
params_beta_0                         0.875911
params_beta_1                         0.981982
params_dropout_rate                   0.337905
params_early_stopping_patience              24
params_embedding_pooling_operation        mean
params_epochs                              116
params_epsilon                        0.000058
params_global_pooling                      max
params_hidden_dimension                    175
params_learning_rate                  0.000194
params_left_stride                          64
params_number_of_hidden_layers               0
params_plateau_divider                       3
params_plateau_patience                     24
params_right_stride                        128
params_thresh

In [28]:
get_best_results(dataset = 'RTSP', gnn = 'GATv2', llm = 'facebook-bart-large', method = 'Surrogate', aggregation = 'No_Aggregation', target_metric = 'accuracy')

number                                      30
value                                 0.908579
params_attention_heads                      15
params_attention_pooling_operation        mean
params_balanced_loss                     False
params_batch_size                          213
params_beta_0                         0.824868
params_beta_1                         0.981183
params_dropout_rate                   0.346246
params_early_stopping_patience              24
params_embedding_pooling_operation         min
params_epochs                               81
params_epsilon                        0.000003
params_global_pooling                     mean
params_hidden_dimension                    194
params_learning_rate                  0.004083
params_number_of_hidden_layers               2
params_plateau_divider                       4
params_plateau_patience                     16
params_threshold                      0.614091
params_weight_decay                   0.000158
user_attrs_ep

## RoBERTa-base, Grouped

In [29]:
get_best_results(dataset = 'SST-2', gnn = 'GATv2', llm = 'FacebookAI-roberta-base', method = 'Grouped', aggregation = 'No_Aggregation', target_metric = 'accuracy')

number                                     104
value                                 0.926606
params_attention_heads                      13
params_attention_pooling_operation        mean
params_balanced_loss                      True
params_batch_size                          197
params_beta_0                         0.823075
params_beta_1                         0.997793
params_dropout_rate                   0.525168
params_early_stopping_patience              11
params_embedding_pooling_operation         max
params_epochs                              178
params_epsilon                        0.000029
params_global_pooling                      max
params_hidden_dimension                    162
params_learning_rate                  0.001031
params_number_of_hidden_layers               1
params_plateau_divider                       9
params_plateau_patience                     23
params_threshold                      0.663891
params_weight_decay                    0.00002
user_attrs_ep

In [30]:
get_best_results(dataset = 'Ohsumed', gnn = 'GATv2', llm = 'FacebookAI-roberta-base', method = 'Grouped', aggregation = 'No_Aggregation', target_metric = 'accuracy')

number                                     134
value                                 0.651157
params_attention_heads                       7
params_attention_pooling_operation         max
params_balanced_loss                     False
params_batch_size                           78
params_beta_0                         0.899905
params_beta_1                         0.990864
params_dropout_rate                   0.432687
params_early_stopping_patience              25
params_embedding_pooling_operation         max
params_epochs                               59
params_epsilon                             0.0
params_global_pooling                     mean
params_hidden_dimension                     88
params_learning_rate                   0.00312
params_left_stride                          32
params_number_of_hidden_layers               1
params_plateau_divider                       5
params_plateau_patience                     17
params_right_stride                        128
params_thresh

In [31]:
get_best_results(dataset = 'R8', gnn = 'GATv2', llm = 'FacebookAI-roberta-base', method = 'Grouped', aggregation = 'No_Aggregation', target_metric = 'accuracy')

number                                     239
value                                 0.960833
params_attention_heads                       4
params_attention_pooling_operation        mean
params_balanced_loss                     False
params_batch_size                          224
params_beta_0                         0.817485
params_beta_1                         0.982257
params_dropout_rate                   0.343777
params_early_stopping_patience              19
params_embedding_pooling_operation        mean
params_epochs                              156
params_epsilon                        0.000046
params_global_pooling                      max
params_hidden_dimension                     89
params_learning_rate                  0.000693
params_left_stride                          32
params_number_of_hidden_layers               0
params_plateau_divider                      10
params_plateau_patience                     16
params_right_stride                        128
params_thresh

In [32]:
get_best_results(dataset = 'RTSP', gnn = 'GATv2', llm = 'FacebookAI-roberta-base', method = 'Grouped', aggregation = 'No_Aggregation', target_metric = 'accuracy')

number                                     229
value                                 0.889592
params_attention_heads                      15
params_attention_pooling_operation        mean
params_balanced_loss                     False
params_batch_size                          110
params_beta_0                         0.871269
params_beta_1                          0.99861
params_dropout_rate                   0.525269
params_early_stopping_patience              15
params_embedding_pooling_operation        mean
params_epochs                              170
params_epsilon                        0.000062
params_global_pooling                      max
params_hidden_dimension                     60
params_learning_rate                   0.00072
params_number_of_hidden_layers               1
params_plateau_divider                      10
params_plateau_patience                     24
params_threshold                      0.826495
params_weight_decay                   0.000054
user_attrs_ep

## RoBERTa-large, Grouped

In [33]:
get_best_results(dataset = 'SST-2', gnn = 'GATv2', llm = 'FacebookAI-roberta-large', method = 'Grouped', aggregation = 'No_Aggregation', target_metric = 'accuracy')

number                                      64
value                                 0.938073
params_attention_heads                      13
params_attention_pooling_operation         max
params_balanced_loss                      True
params_batch_size                          148
params_beta_0                         0.858116
params_beta_1                           0.9938
params_dropout_rate                   0.501638
params_early_stopping_patience              24
params_embedding_pooling_operation         min
params_epochs                              137
params_epsilon                        0.000001
params_global_pooling                     mean
params_hidden_dimension                    101
params_learning_rate                  0.001318
params_number_of_hidden_layers               2
params_plateau_divider                       7
params_plateau_patience                     21
params_threshold                      0.855918
params_weight_decay                   0.000004
user_attrs_ep

In [34]:
get_best_results(dataset = 'Ohsumed', gnn = 'GATv2', llm = 'FacebookAI-roberta-large', method = 'Grouped', aggregation = 'No_Aggregation', target_metric = 'accuracy')

number                                     261
value                                 0.662537
params_attention_heads                      11
params_attention_pooling_operation         max
params_balanced_loss                     False
params_batch_size                           36
params_beta_0                         0.821725
params_beta_1                         0.997318
params_dropout_rate                   0.429354
params_early_stopping_patience              16
params_embedding_pooling_operation        mean
params_epochs                               70
params_epsilon                        0.000049
params_global_pooling                      max
params_hidden_dimension                    197
params_learning_rate                  0.000426
params_left_stride                          32
params_number_of_hidden_layers               1
params_plateau_divider                       9
params_plateau_patience                     11
params_right_stride                        256
params_thresh

In [35]:
get_best_results(dataset = 'R8', gnn = 'GATv2', llm = 'FacebookAI-roberta-large', method = 'Grouped', aggregation = 'No_Aggregation', target_metric = 'accuracy')

number                                     176
value                                 0.970023
params_attention_heads                       4
params_attention_pooling_operation         max
params_balanced_loss                     False
params_batch_size                          156
params_beta_0                         0.865683
params_beta_1                         0.991509
params_dropout_rate                   0.527537
params_early_stopping_patience              16
params_embedding_pooling_operation         max
params_epochs                              198
params_epsilon                             0.0
params_global_pooling                     mean
params_hidden_dimension                    248
params_learning_rate                  0.000936
params_left_stride                         256
params_number_of_hidden_layers               0
params_plateau_divider                       6
params_plateau_patience                     15
params_right_stride                        128
params_thresh

In [36]:
get_best_results(dataset = 'RTSP', gnn = 'GATv2', llm = 'FacebookAI-roberta-large', method = 'Grouped', aggregation = 'No_Aggregation', target_metric = 'accuracy')

number                                     113
value                                 0.890295
params_attention_heads                       4
params_attention_pooling_operation         max
params_balanced_loss                     False
params_batch_size                           91
params_beta_0                         0.858962
params_beta_1                         0.996739
params_dropout_rate                   0.592596
params_early_stopping_patience              12
params_embedding_pooling_operation         min
params_epochs                              191
params_epsilon                        0.000046
params_global_pooling                      max
params_hidden_dimension                    107
params_learning_rate                  0.000506
params_number_of_hidden_layers               1
params_plateau_divider                       3
params_plateau_patience                     22
params_threshold                      0.694255
params_weight_decay                   0.000081
user_attrs_ep

## RoBERTa-large, Surrogate

In [37]:
get_best_results(dataset = 'SST-2', gnn = 'GATv2', llm = 'FacebookAI-roberta-large', method = 'Surrogate', aggregation = 'No_Aggregation', target_metric = 'accuracy')

number                                     185
value                                 0.942661
params_attention_heads                       6
params_attention_pooling_operation        mean
params_balanced_loss                     False
params_batch_size                          231
params_beta_0                         0.879876
params_beta_1                         0.988318
params_dropout_rate                   0.428539
params_early_stopping_patience              15
params_embedding_pooling_operation         max
params_epochs                              162
params_epsilon                             0.0
params_global_pooling                      sum
params_hidden_dimension                    247
params_learning_rate                  0.001466
params_number_of_hidden_layers               4
params_plateau_divider                       8
params_plateau_patience                     11
params_threshold                      0.547045
params_weight_decay                   0.000711
user_attrs_ep

In [38]:
get_best_results(dataset = 'Ohsumed', gnn = 'GATv2', llm = 'FacebookAI-roberta-large', method = 'Surrogate', aggregation = 'No_Aggregation', target_metric = 'accuracy')

number                                     225
value                                 0.651142
params_attention_heads                       5
params_attention_pooling_operation         max
params_balanced_loss                      True
params_batch_size                           80
params_beta_0                         0.867354
params_beta_1                         0.992777
params_dropout_rate                    0.32243
params_early_stopping_patience              24
params_embedding_pooling_operation         min
params_epochs                              135
params_epsilon                             0.0
params_global_pooling                      max
params_hidden_dimension                    236
params_learning_rate                  0.001723
params_left_stride                         128
params_number_of_hidden_layers               0
params_plateau_divider                       8
params_plateau_patience                     15
params_right_stride                        128
params_thresh

In [39]:
get_best_results(dataset = 'R8', gnn = 'GATv2', llm = 'FacebookAI-roberta-large', method = 'Surrogate', aggregation = 'No_Aggregation', target_metric = 'accuracy')

number                                     293
value                                 0.971946
params_attention_heads                       9
params_attention_pooling_operation         max
params_balanced_loss                     False
params_batch_size                          143
params_beta_0                         0.876479
params_beta_1                          0.98383
params_dropout_rate                   0.460693
params_early_stopping_patience              25
params_embedding_pooling_operation         min
params_epochs                              114
params_epsilon                             0.0
params_global_pooling                      max
params_hidden_dimension                    148
params_learning_rate                   0.00032
params_left_stride                         256
params_number_of_hidden_layers               3
params_plateau_divider                       3
params_plateau_patience                     12
params_right_stride                          0
params_thresh

In [40]:
get_best_results(dataset = 'RTSP', gnn = 'GATv2', llm = 'FacebookAI-roberta-large', method = 'Surrogate', aggregation = 'No_Aggregation', target_metric = 'accuracy')

number                                     197
value                                 0.888889
params_attention_heads                      10
params_attention_pooling_operation         max
params_balanced_loss                     False
params_batch_size                          107
params_beta_0                         0.834248
params_beta_1                         0.986524
params_dropout_rate                   0.459045
params_early_stopping_patience              15
params_embedding_pooling_operation        mean
params_epochs                              179
params_epsilon                             0.0
params_global_pooling                      max
params_hidden_dimension                    122
params_learning_rate                  0.001585
params_number_of_hidden_layers               1
params_plateau_divider                       2
params_plateau_patience                     13
params_threshold                      0.767328
params_weight_decay                   0.000013
user_attrs_ep